# Setting up the experiment

In [9]:
#Dependency setup
!python -m pip install numpy matplotlib torch sklearn pandas

You should consider upgrading via the 'c:\Users\leon1\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
# Setting up the files for the first time
!python dataset_setup.py

Files already have been extracted


## Some useful routines

In [12]:
import matplotlib.pyplot as plt
import numpy as np

def display_CIFAR_Image(img):
    img = img.reshape(3,32,32)
    img = np.transpose(img, (1,2,0))
    plt.imshow(img)
    plt.show()

# Setting up a FLWR environment

In [13]:
import flwr as fl
import datetime
import torch
import flwr.server.strategy as strategy

from flwr.common import Metrics
from partition_scripts import partition_CIFAR_equally


today = datetime.datetime.today()
fl.common.logger.configure(identifier="FL Paper Experiment", filename=f"log_FLWR_{today.timestamp()}.txt")
client_resources = None

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

NUM_CLIENTS = 5
TRAINING_ROUNDS = 20

DATA_STORE = {
    "CIFAR10_IID": None,
    "CIFAR10_NonIID": None,
    "CIFAR100_IID": None,
    "CIFAR100_NonIID": None,
    "FedFaces_IID": None,
    "FedFaces_NonIID": None,

}


# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
if DEVICE.type == "cpu":
    client_resources = {"num_cpus": 1, "num_gpus": 0}

Training on cpu using PyTorch 2.2.0+cpu and Flower 1.7.0


Now, we'll set up the Client configurations

In [14]:
from typing import List, Tuple
from logging import DEBUG, INFO
from flwr.common.logger import log
from neural_nets import get_parameters, set_parameters, train, test, VGG7, Net 


class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, cid):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.cid = cid
        self.round = 0

    def get_parameters(self, config):
        # Return the current local parameters
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Train the local model after updating it with the given parameters
        # Return the parameters from the newly trained model, the length
        # of the training data, and a dict (empty in this case)
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs = 1)
        self.round+=1
        log(DEBUG, f"Client {self.cid} in round {self.round}")
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        # Perform the evaluation of the model after updating it with the given
        # parameters. Returns the loss as a float, the length of the validation
        # data, and a dict containing the accuracy
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return loss, len(self.valloader), {'accuracy': float(accuracy)}


def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

Then, setting up the strategies

In [15]:
# Create FedAvg strategy
fedAvg = fl.server.strategy.FedAvg(
    fraction_fit=0.8,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=8,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=10,  # Wait until all 10 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
)
DATA_STORE["CIFAR10_IID"]= partition_CIFAR_equally(NUM_CLIENTS, "CIFAR10")

# A couple of client_fns for using with Flower, one for each dataset experiment
def client_fn_CIFAR10_IID(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Create model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    trainloaders, valloaders,_ =  DATA_STORE["CIFAR10_IID"]
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader, cid).to_client()





A quick run to check if Flower is working ok

In [16]:
# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn_CIFAR10_IID,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=min(TRAINING_ROUNDS, 1)),
    strategy=fedAvg,
    client_resources=client_resources,
)

INFO flwr 2024-02-16 13:45:04,275 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
2024-02-16 13:45:09,556	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-02-16 13:45:11,940 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 2403256320.0, 'memory': 4806512640.0, 'CPU': 12.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-02-16 13:45:11,940 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-02-16 13:45:11,940 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0}
INFO flwr 2024-02-16 13:45:11,956 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 12 actors
INFO flwr 2024-02-16 13:45:11,956 | server.py:89 | Initializing global parameters
INFO flwr 2024-02-16 13:45:11,956 | server.py:276 | Requesting initial parameters from one ra

Ok, so now i'll encapsulate this code to reuse with different strategies and datasets

In [8]:
class FedExperiment():

    def __init__(self, client_fn, datasets, strategy, name="New experiment"):
        self.client_fn = client_fn
        self.datasets = datasets
        self.strategy = strategy
        self.name = name

    def simulate_FL(self):
        log(INFO, "\n" + 10 * "========" + "\n" + self.name + " has started\n" + 10 * "========"  )
        fl.simulation.start_simulation(
                            client_fn=client_fn_CIFAR10_IID,
                            num_clients=NUM_CLIENTS,
                            config=fl.server.ServerConfig(num_rounds=TRAINING_ROUNDS),
                            strategy=fedAvg,
                            client_resources=client_resources,
                        )
        log(INFO, 10 * "========" + "\n" + self.name + " has ended\n" + 10 * "========"  )

In [9]:
exp_CIFAR10_IID = FedExperiment(client_fn=client_fn_CIFAR10_IID, datasets=DATA_STORE["CIFAR10_IID"], strategy=fedAvg, name="CIFAR 10 - IID Distribution")
exp_CIFAR10_IID.simulate_FL()

INFO flwr 2024-02-16 13:39:14,570 | 2254441562.py:10 | 
CIFAR 10 - IID Distribution has started
INFO flwr 2024-02-16 13:39:14,571 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-02-16 13:39:17,435	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-02-16 13:39:19,451 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 2405165875.0, 'memory': 4810331751.0, 'CPU': 12.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-02-16 13:39:19,451 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-02-16 13:39:19,451 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0}
INFO flwr 2024-02-16 13:39:19,451 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 12 actors
INFO flwr 2024-02-16 13:39:19,467 | server.py:89 | Initializing global paramet